<a href="https://colab.research.google.com/github/supercrest/2025-SMS-Essay-Wong-E-Jeh-and-Li-Chang-Cheng/blob/main/Model_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Attribution: A large majority of the code was written with Google Gemini and Microsoft Copilot.

# Transfer Matrix A2
Generation of transfer matrix A2 by inverse-distance weighting.

In [ ]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic

# Define region names and coordinates
region_names = [
    "Jambi", "Riau", "South Sumatra", "West Kalimantan",
    "Central Kalimantan", "Johor", "Singapore"
]

coordinates = [
    (-1.639471, 102.945426),     # Jambi
    (0.500411, 101.547581),      # Riau
    (-3.126684, 104.093055),     # South Sumatra
    (-0.132239, 111.09689),      # West Kalimantan
    (-1.778209, 113.706055),     # Central Kalimantan
    (2.022882, 103.311456),      # Johor
    (1.357107, 103.819499)       # Singapore
]

def build_deposition_matrix(region_names, coordinates, epsilon_km=50):
    n = len(region_names)
    raw_matrix = np.zeros((n, n))

    for i in range(n):  # receptor
        for j in range(n):  # emitter
            d = geodesic(coordinates[i], coordinates[j]).km + epsilon_km
            raw_matrix[i, j] = 1 / d

    row_sums = raw_matrix.sum(axis=1, keepdims=True)
    A = raw_matrix / row_sums
    return pd.DataFrame(A, index=region_names, columns=region_names)

# Generate matrix
A2 = build_deposition_matrix(region_names, coordinates)
print("🌫️ Row-normalized Deposition Matrix A2:\n")
print(A2.round(3))

🌫️ Row-normalized Deposition Matrix A2:

                    Jambi   Riau  South Sumatra  West Kalimantan  \
Jambi               0.598  0.090          0.116            0.031   
Riau                0.091  0.610          0.056            0.027   
South Sumatra       0.124  0.059          0.642            0.036   
West Kalimantan     0.037  0.032          0.040            0.720   
Central Kalimantan  0.030  0.026          0.033            0.096   
Johor               0.061  0.090          0.045            0.029   
Singapore           0.069  0.086          0.050            0.031   

                    Central Kalimantan  Johor  Singapore  
Jambi                            0.024  0.065      0.076  
Riau                             0.021  0.099      0.095  
South Sumatra                    0.028  0.051      0.059  
West Kalimantan                  0.092  0.038      0.041  
Central Kalimantan               0.753  0.029      0.031  
Johor                            0.022  0.558      0.195  
S

In [ ]:
display(A2.round(3))

,Jambi,Riau,South Sumatra,West Kalimantan,Central Kalimantan,Johor,Singapore
Jambi,0.598,0.090,0.116,0.031,0.024,0.065,0.076
Riau,0.091,0.610,0.056,0.027,0.021,0.099,0.095
South Sumatra,0.124,0.059,0.642,0.036,0.028,0.051,0.059
West Kalimantan,0.037,0.032,0.040,0.720,0.092,0.038,0.041
Central Kalimantan,0.030,0.026,0.033,0.096,0.753,0.029,0.031
Johor,0.061,0.090,0.045,0.029,0.022,0.558,0.195
Singapore,0.069,0.086,0.050,0.031,0.023,0.192,0.549


# Model and Simulation
Here is the model, with input vectors E1, E2 used for the simulation.

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# 🌏 Step 1: Regions
regions = [
    "Jambi", "Riau", "South Sumatra", "West Kalimantan",
    "Central Kalimantan", "Johor", "Singapore"
]
n = len(regions)

# 📦 Step 2: Actual emissions (kg CO₂) for E1 (anthropogenic) and E2 (fire)
E1_actual = np.array([9628377621, 17350997505, 22977093926, 14692488527, 7245298531, 35691247384, 49252784000])
E2_actual = np.array([58454785, 54330510, 82593587, 91559754, 126852812, 9078929, 0])

# 💰 Step 3: Abatement cost per kg CO₂
c = np.array([4.665545593, 4.665545593, 4.665545593, 4.665545593, 4.665545593, 2.68347644, 6.627687691])  # calculated values

# 🌫️ Step 4: Deposition matrices A1 and A2 (row-normalized)

A1 = np.full((7,7), 1/7)

# A2 was defined above.

# ⚙️ Step 5: Nonlinear damage parameters
gamma = 1.15  # damage exponent (>1 for convexity)

# 🧮 Step 6: Total cost function with nonlinear damage
def total_cost(x):
    E1_target = x[:n]
    E2_target = x[n:]

    E1_abate = np.clip(E1_actual - E1_target, 0, None)
    E2_abate = np.clip(E2_actual - E2_target, 0, None)
    abate_cost = np.dot(c, E1_abate + E2_abate)

    Q1 = A1 @ E1_target
    Q2 = A2 @ E2_target
    damage = np.sum((Q1 + Q2)**gamma)

    return abate_cost + damage

# 📉 Step 7: Baseline (Nash) costs if no one abates
Q1_base = A1 @ E1_actual
Q2_base = A2 @ E2_actual
baseline_damage = (Q1_base + Q2_base)**gamma
baseline_cost = baseline_damage  # no abatement → only damage

# 🚀 Step 8: Optimization
x0 = np.concatenate([E1_actual, E2_actual])
bounds = [(0, E1_actual[i]) for i in range(n)] + [(0, E2_actual[i]) for i in range(n)]

res = minimize(total_cost, x0, bounds=bounds)
E1_opt = res.x[:n]
E2_opt = res.x[n:]

# 📊 Step 9: Post-optimization calculations
Q1_opt = A1 @ E1_opt
Q2_opt = A2 @ E2_opt
damage_opt = (Q1_opt + Q2_opt)**gamma
abate1 = np.clip(E1_actual - E1_opt, 0, None)
abate2 = np.clip(E2_actual - E2_opt, 0, None)
abatement_cost_opt = c * (abate1 + abate2)
total_cost_by_region = abatement_cost_opt + damage_opt
net_benefit = baseline_cost - total_cost_by_region

# 🧾 Step 10: Display results
df = pd.DataFrame({
    "Region": regions,
    "Actual E1": E1_actual, "Target E1": E1_opt.round(1), "Abate E1": abate1.round(1),
    "Actual E2": E2_actual, "Target E2": E2_opt.round(1), "Abate E2": abate2.round(1),
    "Baseline Cost": baseline_cost.round(2),
    "New Cost": total_cost_by_region.round(2),
    "Net Benefit": net_benefit.round(2)
})

print("🌫️ Cooperative Emission Targets and Participation Check")
display(df)

if (net_benefit < 0).any():
    print("⚠️ Not all regions benefit. Some may refuse to participate.")
else:
    print("✅ All regions have non-negative net benefits. Cooperative solution is stable.")

print(f"\n💰 Total Cooperative Cost: ${res.fun:,.2f}")
print(f"🔥 Total Baseline Cost (no abatement): ${np.sum(baseline_cost):,.2f}")

🌫️ Cooperative Emission Targets and Participation Check


,Region,Actual E1,Target E1,Abate E1,Actual E2,Target E2,Abate E2,Baseline Cost,New Cost,Net Benefit
Jambi,Jambi,9628377621,9.628378e+09,36.4,58454785,58454785.0,0.0,8.019574e+11,8.019574e+11,1324.67
Riau,Riau,17350997505,1.735100e+10,36.4,54330510,54330510.0,0.0,8.016840e+11,8.016840e+11,1324.60
South Sumatra,South Sumatra,22977093926,2.297709e+10,36.4,82593587,82593587.0,0.0,8.025729e+11,8.025729e+11,1324.82
West Kalimantan,West Kalimantan,14692488527,1.469249e+10,36.4,91559754,91559754.0,0.0,8.031579e+11,8.031579e+11,1324.96
Central Kalimantan,Central Kalimantan,7245298531,7.245298e+09,36.4,126852812,126852812.0,0.0,8.042035e+11,8.042035e+11,1325.21
Johor,Johor,35691247384,3.569125e+10,38.4,9078929,9078929.0,0.0,8.005939e+11,8.005938e+11,1391.15
Singapore,Singapore,49252784000,4.925278e+10,34.4,0,0.0,0.0,8.004988e+11,8.004988e+11,1265.91


✅ All regions have non-negative net benefits. Cooperative solution is stable.

💰 Total Cooperative Cost: $5,614,668,422,795.54
🔥 Total Baseline Cost (no abatement): $5,614,668,432,076.85
